## <a id='toc1_1_'></a>[__Пример работы класса `Rag`__](#toc0_)

**Содержание**<a id='toc0_'></a>
- [__Пример работы класса `Rag`__](#toc1_1_)
    - [__Постановка задачи__](#toc1_1_1_)
    - [__Источники текстов__](#toc1_1_2_)
    - [__Инициализация объекта класса `Rag`__](#toc1_1_3_)
    - [__Поиск наиболее близких чанков__](#toc1_1_4_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
### <a id='toc1_1_1_'></a>[__Постановка задачи__](#toc0_)

Дан корпус текстов. Имея модель-эмбеддер и промпт, найти заданное количество фрагментов (текстовых чанков) из корпуса, наиболее близких по заданной метрике.

***
### <a id='toc1_1_2_'></a>[__Источники текстов__](#toc0_)

Здесь используются текстовые файлы, полученные в демонстрации работы класса `DirectoryScanner`.

***
### <a id='toc1_1_3_'></a>[__Инициализация объекта класса `Rag`__](#toc0_)

Параметры (все именованные):

* `source` — файл txt или папка с файлами, среди которых есть файлы txt. Строка `str`. Если источник — папка, то в ней и рекурсивно в ее подпапках будут просмотрены все файлы txt. Мы исходим из того, что если необходимо проанализировать тексты из файлов других форматов и/или расширений, но сначала нужно использовать `DirectoryScanner`.

* `model` — папка с моделью-эмбеддером или имя модели на Hugging Face. Строка `str`.

* `model_max_tokens` — максимальное количество токенов для данной модели-эмбеддера. Целое число `int`. Установка значений меньше паспортного максимума регулирует размер текстовых чанков.

In [1]:
from sberpunk.rag import Rag

In [2]:
rag = Rag(
    source='..\DirectoryScanner\demo files',
    model='BAAI/bge-m3',
    model_max_tokens=512
)

В данном случае мы скачиваем модель с Hugging Face. Строк прогресса `tqdm` здесь нет потому, что ноутбук запускается не первый раз — компоненты модели уже кэшированы.

Коллекция чанков доступна по атрибуту `chunks`. Это список объектов служебного класса `TextChunk`, которые имеют два атрибута:
* `source`, указывающий на файл-источник данного чанка;
* `content`, его текстовое содержание. `content` на самом деле строка. Список строк, который мы видим ниже, — это лишь свертка `textwrap` для компактного удобочитаемого формального и неформального представлений.

Первые три чанка:

In [3]:
rag.chunks[:3]

[{
     "source": "Ленинград - БСЭ.docx.txt",
     "content": [
         "Ленинград [основан 16(27) мая 1703; до 18(31) августа 1914 —",
         "Санкт-Петербург, до 26 января 1924 — Петроград],",
         "промышленный, культурный и научный центр СССР, крупнейший",
         "транспортный узел страны, морской порт. В 1712—28 и",
         "1732—1918 — столица государства, в 1708—1927 — губернский",
         "город, с 1 августа 1927 — центр Ленинградской области РСФСР.",
         "Л. — колыбель Великого Октября, город-герой. Расположен в",
         "западной части Приневской низины, при впадении р. Невы в",
         "Финский залив и на 42 островах разветвленной дельты Невы (в",
         "том числе на островах: Васильевском, Петроградском,",
         "Крестовском, Аптекарском, Елагине, Гутуевском и др.). В",
         "черте города более 40 рек, рукавов, протоков и около 20",
         "искусственных каналов общей протяжённостью свыше 160 км; в",
         "их числе рукава и протоки Невы — 

Чанков всего (из всех просмотренных документов):

In [4]:
len(rag.chunks)

68

***
### <a id='toc1_1_4_'></a>[__Поиск наиболее близких чанков__](#toc0_)

Поиск заданного количества текстовых чанков, наиболее близких по заданной метрике, выполняет метод `get_top_chunks`. Его аргументы:

* `prompt` — промпт, открытый или закрытый вопрос. Открытый начинается с `passage:`, закрытый с `query:`. Строка `str`.

(далее только именованные)

* `metric` — метрика, по которой вычисляется расстояние между промптом и каждым чанком коллекции. Допустимые значения см. [__здесь__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html). Строка `str`. По умолчанию `'cosine'`.

* `n_chunks` — количество чанков. Целое число `int`. По умолчанию `5`.

* `chunk_min_len` — минимальная длина чанка. Чанки меньшей длины игнорируются при формировании топа. Целое число `int`. По умолчанию `0`.

In [5]:
rag.get_top_chunks('passage: условия для отдыха', n_chunks=3)

[{
     "source": "Летний отдых россиян 1.png.txt",
     "content": [
         "Летний сезон - пик туристического отдыха среди россиян.",
         "Дороговизна билетов или проживания - среди ключевых",
         "факторов, которые удерживают россиян от путешествий по",
         "стране летом. Отсутствие уверенности в том, что ожидания",
         "относительно качества сервиса и инфраструктуры места отдыха",
         "совпадут с реальностью — в ТОП-З3 причин, по которым наши",
         "соотечественники остаются дома летом. Также в числе",
         "распространенных барьеров - плохая транспортная доступность",
         "мест для летнего отдыха в стране, неразвитость",
         "инфраструктуры и низкое качество сервиса. Необходимо:",
         "повышать качество инфраструктуры и сервиса в туристических",
         "местах страны, подходящих для летнего отдыха; информировать",
         "об этом в рекламных кампаниях мест отдыха; развивать",
         "региональные транспортные сети, а также с

In [6]:
rag.get_top_chunks('query: какое издательство основал Горький?', n_chunks=1)

[{
     "source": "Ленинград - БСЭ.docx.txt",
     "content": [
         "Первыми советскими издательствами стали основанные в декабре",
         "1917 в Петрограде Литературно-издательский отдел Наркомпроса",
         "и издательство Петроградского совета (ныне Лениздат). В 1918",
         "М. Горьким было организовано издательство «Всемирная",
         "литература». Среди кооперативных издательств, работавших в",
         "Л., наиболее значительны — «Былое» (1917—1925), «Academia»",
         "(1922—28, с 1929 в Москве), «Время» (1922—34), «Издательство",
         "писателей в Ленинграде» (1927—38). Ныне Л. — один из",
         "крупнейших центров издательского дела и полиграфической",
         "промышленности СССР. Здесь находится свыше 30 издательств, в",
         "том числе центральные издательства «Аврора»,",
         "Гидрометеоиздат, «Судостроение», республиканское",
         "издательство «Художник РСФСР», Лениздат; отделения крупных",
         "издательств, базирующихся в Моск